In [1]:
import networkx as nx
from node2vec import Node2Vec
import pandas as pd
from tqdm import tqdm

c:\Users\drago\miniconda3\envs\node2vec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
outfit = pd.read_csv("./outfit_2.csv")
outfit_map = {v:i for i, v in enumerate(outfit['outfit_id'].unique())}
outfit['outfit_id'] = outfit['outfit_id'].map(outfit_map)
outfit['tags'] = outfit['tags'].apply(lambda x: set(x.split(",")))
outfit

,outfit_id,gender,style,season,tags,new_style,style_id
0,0,F,스트릿,가을,"{스트릿, 가을, 반팔티셔츠}",F_스트릿,0
1,1,F,캐주얼,봄,"{스니커즈, 봄, 캐주얼, 나이키}",F_캐주얼,1
2,2,F,캐주얼,여름,"{캐주얼, 여름, 레더스커트, 셔츠, 워커}",F_캐주얼,1
3,3,F,로맨틱,여름,"{트랙팬츠, 스니커즈, 나이키, 슬리브리스, 여름, 로맨틱, 숄더백}",F_로맨틱,2
4,4,M,스트릿,여름,"{스트릿, 스니커즈, 베스트, 나이키, 여름, 데님팬츠, 비니, 크로스백}",M_스트릿,3
...,...,...,...,...,...,...,...
10424,10424,M,스트릿,여름,"{스트릿, 여름}",M_스트릿,3
10425,10425,F,걸리시,여름,"{여름, 걸리시}",F_걸리시,4
10426,10426,F,스트릿,여름,"{스트릿, 여름}",F_스트릿,0
10427,10427,F,스트릿,여름,"{스트릿, 여름}",F_스트릿,0


In [3]:
# male = outfit[outfit['gender']=='M'].copy()
# female = outfit[outfit['gender']=='F'].copy()

In [4]:
# male.info()

In [5]:
def jaccard_similarity(tags1: set, tags2: set):
    # tags1 = set(tags1.split(","))
    # tags2 = set(tags2.split(","))
    intersection = tags1.intersection(tags2)
    union = tags1.union(tags2)
    similarity = len(intersection) / len(union)
    return similarity

In [6]:
# jaccard_similarity(outfit.loc[10426, 'tags'], outfit.loc[10427, 'tags'])

In [7]:
def get_sim_df(df: pd.DataFrame):
    n = len(df)
    result = list()

    for i in tqdm(range(n-1)):
        for j in range(i+1, n):
            similarity = jaccard_similarity(df.iloc[i, 4], df.iloc[j, 4])
            result.append([df.iloc[i, 0], df.iloc[j, 0], similarity])
        
    sim_df = pd.DataFrame(result, columns=['id_1', 'id_2', 'similarity'])
    
    return sim_df

In [8]:
# male_sim = get_sim_df(male)

In [9]:
# male_sim.to_csv("./male_sim.csv", index=False)

In [10]:
# female_sim = get_sim_df(female)

In [11]:
# female_sim.to_csv("./female_sim.csv", index=False)

In [12]:
male_sim = pd.read_csv("./male_sim.csv")
male_sim

,id_1,id_2,similarity
0,4,6,0.300000
1,4,7,0.666667
2,4,8,0.250000
3,4,10,0.250000
4,4,12,0.300000
...,...,...,...
4643623,10406,10414,0.333333
4643624,10406,10424,0.333333
4643625,10410,10414,0.333333
4643626,10410,10424,1.000000


In [17]:
male_sim = male_sim[male_sim['similarity']!=0]
male_sim

,id_1,id_2,similarity
0,4,6,0.300000
1,4,7,0.666667
2,4,8,0.250000
3,4,10,0.250000
4,4,12,0.300000
...,...,...,...
4643623,10406,10414,0.333333
4643624,10406,10424,0.333333
4643625,10410,10414,0.333333
4643626,10410,10424,1.000000


In [18]:
male_graph = nx.Graph()
for row in male_sim.values:
    male_graph.add_edge(int(row[0]), int(row[1]), weight=row[2])   

In [19]:
male_graph.number_of_nodes()

3048

In [20]:
male_graph.number_of_edges()

2742170

In [22]:
node2vec = Node2Vec(male_graph, dimensions=64, walk_length=5, num_walks=50, workers=4)

Computing transition probabilities:   6%|▌         | 177/3048 [32:55<8:53:59, 11.16s/it] 


KeyboardInterrupt: 